# Executing Matlab code using Octave through python

In [1]:
! conda list octave

# packages in environment at /usr/local/share/miniconda3/envs/env:
#
# Name                    Version                   Build  Channel
octave                    7.3.0                h4bc5dab_3    conda-forge
octave_kernel             0.35.1             pyhd8ed1ab_0    conda-forge


In [2]:
import subprocess

In [6]:
import oct2py

In [7]:
oc = oct2py.Oct2Py()

In [8]:
x = oc.zeros(3, 3)

In [9]:
print(x)

[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


In [11]:
from oct2py import octave

In [17]:
import os

In [29]:
oc.disp('Hola mundo')

'Hola mundo\n'

In [30]:
octave.disp('Hola mundo')

'Hola mundo\n'

In [32]:
oc.source('./octave-tests/hola_mundo.m')




Hola Mundo


In [27]:
! ls ./octave-tests

hola_mundo.m


In [28]:

path = os.path.expanduser("./octave-tests")
octave.addpath(path)  # doctest: +SKIP
octave.addpath(octave.genpath(path)) 
#octave.run("hola_mundo.m")
octave.source("hola_mundo.m")

Oct2PyError: Octave evaluation error:
error: source: error sourcing file '/home/macu/work/nbs_pipeline/hola_mundo.m'

In [4]:
# Crear una instancia de Oct2Py
with octpy.Oct2Py() as oc:
    # Ejecutar un comando de Octave directamente
    oc.eval('disp("¡Hola, Mundo!")')

Oct2PyError: Octave evaluation error:
parse error:

  syntax error



In [ ]:
# Comando para ejecutar el script de Octave
# Asegúrate de que la ruta al script de Octave sea accesible desde donde estás ejecutando este script de Python
command = ["octave-cli", "hello_world.m"]

In [ ]:

# Ejecuta el comando y captura la salida
result = subprocess.run(command, capture_output=True, text=True)

# Imprime la salida estándar (incluyendo "Hola Mundo")
print("Salida estándar:")
print(result.stdout)
# Imprime el error estándar si existe alguno
if result.stderr:
    print("Error estándar:")
    print(result.stderr)
